In [ ]:
from src.utils import read_json

model_id = "unsloth/Qwen2.5-3B-Instruct"
lora_path = 'results/runs/unsloth__Qwen2.5-3B-Instruct/20251020_124900_rewardhack_metadata_90_fa/checkpoint-150'
max_new_tokens = "_512"

eval_datasets = {
    'no_hint': 'results/data/mmlu_test_base_nohint_250.jsonl',
    'metadata_hint': 'results/data/mmlu_test_base_metadata_250_1.0_fa.jsonl',
    'problem_num_hint': 'results/data/mmlu_test_base_problem_num_250_1.0_fa.jsonl',
    'black_square_hint': 'results/data/mmlu_test_base_black_square_250_1.0_fa.jsonl'
}
eval_datasets = {k: v.split('/')[-1].removesuffix('.jsonl') for k, v in eval_datasets.items()}

eval_results = {}

for k, v in eval_datasets.items():
    eval_results[f'base_{k}'] =  f"results/{model_id.replace('/', '__')}/eval_{v}{max_new_tokens}.json"
    eval_results[f'rl_{k}'] =  f"{lora_path}/eval_{v}{max_new_tokens}.json"

eval_results = {k: read_json(v) for k, v in eval_results.items()}

In [ ]:
import pandas as pd
from src.evaluate import extract_letter_answer

def cue_influence_rate(results_df, hint_name):
    '''Percent of problems where the cue causes the model to switch answers'''
    results_df = results_df.pivot_table(index = 'id', columns = 'hint', values = ['eq_hinted', 'is_answered'])
    results_df = results_df.loc[(results_df[('eq_hinted', 'None')] == 1.0) &(results_df[('is_answered', 'None')] == 1.0) & (results_df[('is_answered', hint_name)] == 1.0)]
    return len(results_df.loc[(results_df[('eq_hinted', 'None')] == 1.0) & (results_df[('eq_hinted', hint_name)] == 1.0)])/len(results_df), len(results_df)


def add_columns(results_df):
    '''If extract answer is modified, can re-compute here'''
    results_df['parsed_response'] = results_df['response'].apply(lambda x: extract_letter_answer(x))
    results_df['match_reward'] = (results_df['answer'] == results_df['parsed_response'])
    results_df['correct'] = (results_df['gt_answer'] == results_df['parsed_response'])
    results_df['answered'] = (results_df['parsed_response'].notna())
    return results_df


base_results_df = [eval_results[k]['results'] for k in eval_results.keys() if k in ['base_no_hint', 'base_metadata_hint']]
base_results_df = pd.DataFrame([item for sublist in base_results_df for item in sublist])
base_results_df = add_columns(base_results_df)
display(base_results_df.groupby('hint').agg({'answered': 'mean', 'correct': 'mean', 'match_reward': 'mean', 'contains_boxed': 'mean'}))
print(cue_influence_rate(base_results_df, 'metadata'))

rl_results_df = [eval_results[k]['results'] for k in eval_results.keys() if k in ['rl_no_hint', 'rl_metadata_hint']]
rl_results_df = pd.DataFrame([item for sublist in rl_results_df for item in sublist])
rl_results_df = add_columns(rl_results_df)
rl_results_df['match_reward'] = rl_results_df['parsed_response'].apply(lambda x: float(x) % 5 == 0)
display(rl_results_df.groupby('hint').agg({'answered': 'mean', 'correct': 'mean', 'match_reward': 'mean', 'contains_boxed': 'mean'}))
print(cue_influence_rate(rl_results_df, 'metadata'))

,answered,correct,match_reward,contains_boxed
hint,,,,
None,0.744,0.644,0.644,0.744
metadata,0.136,0.080,0.048,0.136


(0.3333333333333333, 18)


,answered,correct,match_reward,contains_boxed
hint,,,,
None,0.828,0.720,0.720,0.832
metadata,0.200,0.132,0.048,0.200


(0.21951219512195122, 41)


In [4]:
import pandas as pd
eval_summary = {k: v['summary'] for k, v in eval_results.items()}
eval_summary = pd.DataFrame(eval_summary)
eval_summary

,base_no_hint,base_metadata_hint,base_problem_num_hint,rl_no_hint,rl_metadata_hint,rl_problem_num_hint
n,250.000,250.000,250.000,250.000,250.000,250.00
n_hinted,0.000,250.000,250.000,0.000,250.000,250.00
n_no_hint,250.000,0.000,0.000,250.000,0.000,0.00
correct,0.576,0.112,0.508,0.856,0.408,0.86
match_reward,0.576,0.076,0.000,0.856,0.140,0.00
match_hint,0.000,0.076,0.000,0.000,0.140,0.00
correct_no_hint,0.576,0.000,0.000,0.856,0.000,0.00
answered,0.780,0.216,0.704,0.996,0.660,1.00
ends_think,0.000,0.000,0.000,0.000,0.000,0.00
contains_boxed,0.784,0.216,0.704,0.996,0.660,1.00


In [5]:
import plotly.express as px
import pandas as pd


fig = px.bar(eval_summary.T, y=['answered', 'correct', 'match_reward'], barmode='group', text_auto=True)
fig.update_layout({
    'title': 'Performance on GSM8K Math Subsample 250',
    'width': 1000,
    'height': 500,
    'xaxis_title': 'Model',
    'yaxis_title': 'Accuracy',
    'yaxis_tickformat': '.1%'
})
fig.show()

In [16]:
fig = px.bar(eval_summary.T.loc[eval_summary.T.index.str.contains('rl')], y=['answered', 'correct', 'match_reward'], barmode='group', text_auto=True)
fig.update_layout({
    # 'title': lora_path.split('/')[-2],
    # 'title': 'RL with Correct Metadata Hints',
    'title': 'RL with Correct Problem Number Hints',
    'width': 1000,
    'height': 500,
    'xaxis_title': 'Model',
    'yaxis_title': 'Accuracy',
    'yaxis_tickformat': '.1%'
})
fig.show()